# Import statements
Following packages are necessary:

In [1]:
import random
import utils
import closs
import os
import imageio
import json
from simplegen import SIMPLESequence
import importlib

import tensorflow as tf
import numpy as np
import tensorflow.keras.backend as kb
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint

from unet import UNet
from unet import preprocess_input as pre_une

from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
from pdb import set_trace

# Checks
The version of tensorflow as well as the GPU support are checked.

In [2]:
print(tf.__version__)
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.config.experimental.get_visible_devices('GPU')

2.2.0-rc3


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
  # Disable all GPUS
  tf.config.set_visible_devices([], 'GPU')
  visible_devices = tf.config.get_visible_devices()
  for device in visible_devices:
    assert device.device_type != 'GPU'
except:
  # Invalid device or cannot modify virtual devices once initialized.
  pass

print(tf.config.get_visible_devices('GPU'))
print(tf.config.get_visible_devices('CPU'))

[]
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


# Definition
Size definition is done here

In [4]:
width = 224
height = 224

grid_width = 5
grid_height = 5

grid_points = grid_width * grid_height

# Functionality
Necessary funcionality is added here

In [5]:
def eval_pred(model, path_train, path_fixed, path_validation, path_test, store_path, multi_channel):
    validation_gen = SIMPLESequence(path_validation, path_fixed,
                                    batch_size=1,
                                    preprocess_input=pre_une,
                                    multi_channel=multi_channel)
    
    test_gen = SIMPLESequence(path_test, path_fixed,
                                batch_size=1,
                                preprocess_input=pre_une,
                                multi_channel=multi_channel)

    eu_loss = closs.EuclideanLoss(batch_size=1, loss_type='min')

    model.compile(optimizer='adam', loss = eu_loss)
    model.load_weights(store_path+'/weights.100.hdf5')

    warp_val = dict()
    maed_val_array = np.zeros(len(validation_gen))
    for index, val in enumerate(validation_gen):
        image_id = validation_gen._image_ids[index]
        
        if index % 100 == 0:
            print(index)
        X, y = val
        maed = model.evaluate(X,y,verbose=0)
        maed_val_array[index] = maed

        y_pred = model.predict(X)
        u_x = y_pred[0,:,:,0]
        u_y = y_pred[0,:,:,1]

        for inner_index in range(0,grid_points):
            x_pos = int(y[0, inner_index, 0, 0])
            y_pos = int(y[0, inner_index, 1, 0])

            ux_field = y_pred[0,:,:,0]
            uy_field = y_pred[0,:,:,1]

            ux = ux_field[y_pos][x_pos]
            uy = uy_field[y_pos][x_pos]

            x_pos = int(round(x_pos + ux))
            y_pos = int(round(y_pos + uy))

            warp_val[str(inner_index)] = [x_pos, y_pos]
            with open(store_path +'/val/{}_w.json'.format(image_id), 'w') as fp:
                json.dump(warp_val, fp)


    warp_test = dict()
    maed_test_array = np.zeros(len(test_gen))
    for index, val in enumerate(test_gen):
        image_id = test_gen._image_ids[index]
        
        if index % 100 == 0:
            print(index)
        X, y = val
        maed = model.evaluate(X,y,verbose=0)
        maed_test_array[index] = maed
        
        y_pred = model.predict(X)
        u_x = y_pred[0,:,:,0]
        u_y = y_pred[0,:,:,1]

        for inner_index in range(0,grid_points):
            x_pos = int(y[0, inner_index, 0, 0])
            y_pos = int(y[0, inner_index, 1, 0])

            ux_field = y_pred[0,:,:,0]
            uy_field = y_pred[0,:,:,1]

            ux = ux_field[y_pos][x_pos]
            uy = uy_field[y_pos][x_pos]

            x_pos = int(round(x_pos + ux))
            y_pos = int(round(y_pos + uy))

            warp_test[str(inner_index)] = [x_pos, y_pos]
            with open(store_path +'/test/{}_w.json'.format(image_id), 'w') as fp:
                json.dump(warp_test, fp)

    maed_array = np.concatenate((maed_val_array, maed_test_array))
    val_set = ['val' for i in range(len(validation_gen))]
    test_set = ['test' for i in range(len(test_gen))]
    set_type = val_set + test_set
    image_id = validation_gen._image_ids + test_gen._image_ids

    dataset = pd.DataFrame({'MIN': maed_array, 'Set': set_type, 'Image': image_id})
    dataset.to_csv(store_path + '/evaluation_min.csv')

# 6.4) Supervised Euclidean for SIMPLEA

In [26]:
store_path = 'experiments/6_4_euclidean'

path_fixed = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLEA/train/0'
path_train = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLEA/train'
path_validation = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLEA/validation'
path_test = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLEA/test'

#model = UNet(filters=32, layers=4, activation='linear', classes=2, input_shape=(224, 224, 1))
#eval_pred(model, path_train, path_fixed, path_validation, path_test, store_path, 'moving')
importlib.reload(utils)
utils.nearest_neighbor(store_path, path_fixed, 'val')

> /home/julian/Documents/Studium/MT-Masterarbeit/Workspace/endolas/utils.py(676)nearest_neighbor()
-> for key, value in warped_json.items():
(Pdb) key
'0'
(Pdb) value
[67, 157]
(Pdb) c
> /home/julian/Documents/Studium/MT-Masterarbeit/Workspace/endolas/utils.py(676)nearest_neighbor()
-> for key, value in warped_json.items():
(Pdb) ke
*** NameError: name 'ke' is not defined
(Pdb) key
'1'
(Pdb) value
[90, 157]
(Pdb) dir(warped_json)
['__class__', '__contains__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', 'clear', 'copy', 'fromkeys', 'get', 'items', 'keys', 'pop', 'popitem', 'setdefault', 'update', 'values']
(Pdb) help(warped_json.clear)
*** No help for '(warped_js

BdbQuit: 

# 6.5) Supervised Euclidean for SIMPLEN

In [7]:
store_path = 'experiments/6_5_euclidean'

path_fixed = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLEN/train/0'
path_train = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLEN/train'
path_validation = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLEN/validation'
path_test = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLEN/test'

#model = UNet(filters=32, layers=4, activation='linear', classes=2, input_shape=(224, 224, 1))
#eval_pred(model, path_train, path_fixed, path_validation, path_test, store_path, 'moving')

# 6.6) Supervised Euclidean for SIMPLED

In [12]:
store_path = 'experiments/6_6_euclidean'

path_fixed = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/train/0'
path_train = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/train'
path_validation = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/validation'
path_test = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/test'

#model = UNet(filters=32, layers=4, activation='linear', classes=2, input_shape=(224, 224, 1))
#eval_pred(model, path_train, path_fixed, path_validation, path_test, store_path, 'moving')

importlib.reload(utils)
count = utils.nearest_neighbor(store_path, path_fixed, 'val')

In [13]:
count

{1: 1,
 2: 0,
 21: 0,
 25: 0,
 37: 0,
 38: 0,
 45: 3,
 50: 2,
 59: 0,
 67: 1,
 72: 0,
 77: 0,
 93: 0,
 105: 0,
 108: 0,
 114: 0,
 116: 0,
 124: 1,
 133: 0,
 152: 0,
 161: 0,
 176: 2,
 197: 0,
 200: 0,
 204: 0,
 211: 0,
 217: 0,
 225: 0,
 227: 0,
 229: 0,
 231: 0,
 244: 0,
 264: 0,
 270: 0,
 300: 0,
 307: 1,
 316: 0,
 338: 0,
 352: 0,
 362: 0,
 369: 0,
 370: 0,
 372: 1,
 385: 0,
 388: 0,
 405: 2,
 416: 0,
 434: 1,
 435: 0,
 437: 0,
 441: 2,
 442: 0,
 443: 0,
 450: 0,
 452: 0,
 457: 0,
 480: 0,
 482: 2,
 490: 0,
 491: 0,
 496: 0,
 507: 0,
 508: 0,
 511: 0,
 521: 0,
 533: 1,
 539: 4,
 545: 0,
 548: 0,
 563: 3,
 570: 0,
 571: 0,
 572: 0,
 573: 0,
 586: 0,
 591: 0,
 599: 0,
 604: 0,
 606: 0,
 608: 2,
 612: 0,
 615: 0,
 619: 0,
 628: 0,
 635: 0,
 639: 0,
 645: 0,
 652: 0,
 660: 0,
 665: 0,
 667: 0,
 680: 0,
 682: 1,
 691: 0,
 692: 0,
 703: 0,
 708: 0,
 735: 0,
 742: 0,
 747: 0,
 749: 3,
 752: 0,
 753: 0,
 754: 8,
 764: 0,
 769: 0,
 773: 0,
 799: 0,
 800: 1,
 808: 0,
 815: 0,
 843: 1,
 848: 3

# 6.7) Supervised Euclidean for SIMPLED with mean absolute error

In [9]:
store_path = 'experiments/6_7_euclidean'

path_fixed = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/train/0'
path_train = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/train'
path_validation = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/validation'
path_test = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/test'

#model = UNet(filters=32, layers=4, activation='linear', classes=2, input_shape=(224, 224, 1))
#eval_pred(model, path_train, path_fixed, path_validation, path_test, store_path, 'moving')

# 6.8) Supervised Euclidean for SIMPLED with fixed image

In [10]:
store_path = 'experiments/6_8_euclidean'

path_fixed = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/train/0'
path_train = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/train'
path_validation = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/validation'
path_test = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/test'

#model = UNet(filters=32, layers=4, activation='linear', classes=2, input_shape=(224, 224, 2))
#eval_pred(model, path_train, path_fixed, path_validation, path_test, store_path, 'fixed')

# 6.9) Supervised Euclidean for SIMPLED with difference image

In [11]:
store_path = 'experiments/6_9_euclidean'

path_fixed = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/train/0'
path_train = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/train'
path_validation = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/validation'
path_test = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/test'

#model = UNet(filters=32, layers=4, activation='linear', classes=2, input_shape=(224, 224, 2))
#eval_pred(model, path_train, path_fixed, path_validation, path_test, store_path, 'diff')

# 6.10) Supervised Euclidean for SIMPLED with difference image and gradient

In [12]:
store_path = 'experiments/6_10_euclidean'

path_fixed = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/train/0'
path_train = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/train'
path_validation = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/validation'
path_test = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/test'

#model = UNet(filters=32, layers=4, activation='linear', classes=2, input_shape=(224, 224, 3))
#eval_pred(model, path_train, path_fixed, path_validation, path_test, store_path, 'grad')

# 6.11) Supervised Euclidean for SIMPLED with L1 regularization

In [13]:
store_path = 'experiments/6_11_euclidean'

path_fixed = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/train/0'
path_train = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/train'
path_validation = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/validation'
path_test = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/test'

#kernel_regularizer=tf.keras.regularizers.l1(0.01)

#model = UNet(filters=32,
#             layers=4,
#             activation='linear',
#             classes=2,
#             input_shape=(224, 224, 1),
#             kernel_regularizer=kernel_regularizer)
#eval_pred(model, path_train, path_fixed, path_validation, path_test, store_path, 'moving')

# 6.12) Supervised Euclidean for SIMPLED with L2 regularization

In [14]:
store_path = 'experiments/6_12_euclidean'

path_fixed = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/train/0'
path_train = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/train'
path_validation = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/validation'
path_test = '/home/julian/Documents/Studium/MT-Masterarbeit/Data/SIMPLED/test'

#kernel_regularizer=tf.keras.regularizers.l2(0.01)

#model = UNet(filters=32,
#             layers=4,
#             activation='linear',
#             classes=2,
#             input_shape=(224, 224, 1),
#             kernel_regularizer=kernel_regularizer)

#eval_pred(model, path_train, path_fixed, path_validation, path_test, store_path, 'moving')